In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIXED #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIXED change for your username and password and CRUD Python module name
#username = "bstaton"
#password = "aacAdmin"
shelter = AnimalShelter('bstaton', 'aacAdmin')


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIXED Add in Grazioso Salvare’s logo
image_filename = 'Grazioso-Salvare-Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIXED Place the HTML image tag in the line below into the app.layout code according to your design
#FIXED Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H3('Byron Dewayne Staton - SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         style={'height':'10%', 'width':'10%'})),
    
    html.Hr(),
    html.Div(
    #FIXED Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value' : 'reset'},
            ],
            value='reset',
            labelStyle={'display':'inline-block'}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXED: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = False,
        fixed_columns={'headers':True, 'data':1},
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={'minWidth': '100%'}
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(value):
### FIXED Add code to filter interactive data table with MongoDB queries
    if (value == 'water'):
        # Water rescue dogs
        df = pd.DataFrame.from_records(list(shelter.read_all({'animal_type':'Dog',
                                                          'breed':{
                                                              '$in':[
                                                                  'Chesa Bay Retr Mix',
                                                                  'Labrador Retriever Mix',
                                                                  'Newfoundland Mix']
                                                          },'sex_upon_outcome':'Intact Female', 
                                                          'age_upon_outcome_in_weeks':{'$gte':26}, 
                                                          'age_upon_outcome_in_weeks':{'$lte':156}})))
    elif (value == 'mountain'):
        # Mountain and wilderness rescue dogs
        df = pd.DataFrame.from_records(list(shelter.read_all({'animal_type':'Dog',
                                                          'breed':{
                                                              '$in':[
                                                                  'Alaskan Malamute',
                                                                  'German Shepard',
                                                                  'Old English Sheepdog',
                                                                  'Rottweiler',
                                                                  'Siberian Husky']
                                                          },'sex_upon_outcome':'Intact Male',
                                                          'age_upon_outcome_in_weeks':{'$gte':26}, 
                                                          'age_upon_outcome_in_weeks':{'$lte':156}})))
    elif (value == 'disaster'):
        # Disaster rescue dogs
        df = pd.DataFrame.from_records(list(shelter.read_all({'animal_type':'Dog',
                                                          'breed':{
                                                              '$in':[
                                                                  'Bloodhound',
                                                                  'Doberman Pinscher',
                                                                  'German Shepard',
                                                                  'Golden Retriever',
                                                                  'Rottweiler']
                                                          },'sex_upon_outcome':'Intact Male',
                                                          'age_upon_outcome_in_weeks':{'$gte':26},
                                                          'age_upon_outcome_in_weeks':{'$lte':156}})))
    else:
        # Reset
        df = pd.DataFrame.from_records(shelter.read_all({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    ###FIXED ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    loc = [dff.location_lat[0], dff.location_long[0]]
    
    # Austin, TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=loc, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tooltip and popup
            dl.Marker(position=loc, children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.name[0])
                ])
            ])
        ])
    ]



app